In [ ]:
%config InlineBackend.figure_format = 'retina'
import os
import numpy as np
import matplotlib.pyplot as plt
import globhunt as gh
plt.style.use('jpg.mplstyle')

# `MultiBandImage`
- Images are stored as `MultiBandImage` (`mbi`) objects
- You can generate `MultiBandImage` objects two ways

# Making `MultiBandImage` objects

### 1) `HSCButler`
- The `butler` fetches images and psfs from the HSC server. You need a username and password. 

In [ ]:
butler = gh.HSCButler('grecoj')

In [ ]:
%%time

# this is a helper function to get parameters for 
# galaxies in the HSC LSB galaxy sample
params = gh.get_hsc_lsbg_params(490) 

# here's how you fetch a cutout
width = 60 # arcsec
mbi = butler.fetch_cutout(params.ra, params.dec, width)

---
Using the `butler` is great because you can grab data from any position within the HSC footprint. 

But the downside is that it can be quite slow, depending on your internet speed. 

### 2) Storing the data locally and reading into a `file_dict`

In [ ]:
# If you make your own file dictionary, you can use this method
gh.MultiBandImage.from_file_dict?

In [ ]:
# If you follow a particular directory structure, you can use 
# this helper functions 
file_dict = gh.make_file_dict(490, 'griz', '/Users/jgreco/Desktop/globhunt-data/')
file_dict

In [ ]:
mbi = gh.MultiBandImage.from_file_dict(file_dict)

In [ ]:
# There's a helper function to do this in one step
mbi = gh.fetch_multiband_image(490, 'griz', '/Users/jgreco/Desktop/globhunt-data/', width=60)

## `MultiBandImage` objects
- have attributes for the object image, variance (var), psf, and HSC mask
- you can see all the attributes and methods using tab complete

In [ ]:
# the image attributes store the data for all the bands, which you can access as 
# attributes or dictionary keys

# as attributes: mbi.image.i, image.var.i, image.psf.i, image.hsc_mask.i
# as dictionary keys: mbi.image['i'], image.var['i'], image.psf['i'], image.hsc_mask['i']

# there's a viz module for storing plotting functions.
gh.show_image(mbi.image.i)
gh.show_image(mbi.var.i)
gh.show_image(mbi.hsc_mask.i);

In [ ]:
# mbi objects have a method for making rgb images
rgb = mbi.make_rgb(stretch=0.4)
gh.show_image(rgb);

# The `PSFPhotometry` pipeline
The pipeline is initialized with:
- the output directory for imfit, which is mostly temporary files
- (optionally) the pipeline configuration 

In [ ]:
# you can pass the config as a dictionary or a yaml file
pipe = gh.PSFPhotometry('/Users/jgreco/local-io/imfit-io/', '../configs/pipe-config.yml')

In [ ]:
%%time
# here's how you run the pipeline with the loaded configuration.
# the second two parameters tell the pipeline to save residual images
# from the psf photometry (save_residual_images) 
# and the imfit sersic fits (save_viz), which will save png images to the
# directory you gave for imfit input/output
cat = pipe.run_pipeline(mbi, save_residual_images=True, save_viz=True)

In [ ]:
# the pipeline also has a make_rgb method. 
# you need to pass the type of image you want
res_rgb = pipe.make_rgb('residual_image_forced', stretch=0.4)
gh.show_image(res_rgb);

# Saving Sersic models and `mbi` objects
- This is very helpful when you are testing or measuring completeness and want to run the pipeline many times

In [ ]:
# the sersic model is saved in binary in .npy format
sersic_model_fn = '/Users/jgreco/Desktop/dev-gh/G490-sersic-model.npy'
pipe.sersic_fitter.save_model(sersic_model_fn)

In [ ]:
# then you just need to pass the file name to the run_pipeline method
# notice it skips the imfit step
cat = pipe.run_pipeline(mbi, sersic_model=sersic_model_fn)

In [ ]:
# you can save mbi objects as pickle files
# this is useful if you want fast access to files 
# you downloaded files using the hsc butler
mbi_pkl = '/Users/jgreco/Desktop/dev-gh/G90-mbi.pkl'
mbi.to_pickle(mbi_pkl)

In [ ]:
# to read in a saved mbi object, use the from_pickle method
mbi = gh.MultiBandImage.from_pickle(mbi_pkl)

# The `synths` module

In [ ]:
# inject synthetic point sources into images using th inject function
synth_catalog, mbi_synths = gh.synths.inject(mbi, mag_range=[23, 28], nstars=300, edge_buffer=10)

In [ ]:
gh.show_image(mbi_synths.make_rgb(stretch=0.4));

In [ ]:
# treat the image with mock stars the same 
cat = pipe.run_pipeline(mbi_synths, sersic_model=sersic_model_fn, save_residual_images=True)

In [ ]:
res_rgb = pipe.make_rgb('residual_image_forced', stretch=0.4)
gh.show_image(res_rgb);

In [ ]:
# the synth module has a helper function to find the sources you recovered
injected, recovered = gh.synths.recovery(cat, synth_catalog)

# Completeness
- There are a couple functions for plotting the artificial star test results

In [ ]:
gh.plot_completeness(synth_catalog, injected, 'i', xlim=[22.5, 28]);

In [ ]:
gh.plot_measurement_offset(injected, recovered, 'i', xlim=[23, 28]);